# ⚙️ Setup

In [52]:
# from google.colab import drive
# drive.mount('/content/drive')

In [53]:
localpath='data'

In [54]:
!pip install -U transformers
!pip install bitsandbytes accelerate
!pip install Whoosh # search engine library

You should consider upgrading via the '/Users/mariaribalta/Documents/EHU/Applications_2/final_project/apps2_project_venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mariaribalta/Documents/EHU/Applications_2/final_project/apps2_project_venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mariaribalta/Documents/EHU/Applications_2/final_project/apps2_project_venv/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
!pip install mistralai

In [55]:
# !pip install colbert-ai[torch,faiss-gpu]

In [56]:
import pandas as pd
from pprint import pprint
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [57]:
from groq import Groq
from dotenv import load_dotenv
import os
import json

In [58]:
import codecs # base classes for standard Python codecs, like text encodings (UTF-8,...)
from IPython.core.display import display, HTML # object displaying in different formats
from whoosh.index import * # whoosh: full-text indexing and searching
from whoosh.fields import *
from whoosh import qparser
import glob
import random
import torch
import numpy as np
import os

/var/folders/b5/zsgbxty17rn21slqnmj3c2t00000gn/T/ipykernel_19094/1853117247.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML # object displaying in different formats


In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

/Users/mariaribalta/Documents/EHU/Applications_2/final_project/apps2_project_venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mariaribalta/Documents/EHU/Applications_2/final_project/apps2_project_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import os
import pandas as pd

# 📚 Data Extraction

In [59]:
!git clone https://github.com/rubzip/LyricGenerator.git

fatal: destination path 'LyricGenerator' already exists and is not an empty directory.


In [60]:
datapath = 'LyricGenerator/data/datasets'
train = pd.read_csv(f'{datapath}/train.csv')
dev = pd.read_csv(f'{datapath}/val.csv')
test = pd.read_csv(f'{datapath}/test.csv')

In [61]:
db_stored_info_fn = lambda row: f"artist: {row['artist']}, title: {row['song']}, tag: {row['tag_2']}, content: {row['content']}"

In [62]:
train['db_stored_info'] = train.apply(db_stored_info_fn, axis=1)
dev['db_stored_info'] = dev.apply(db_stored_info_fn, axis=1)

In [63]:
train.head()

,artist,song,tag,content,tag_2,db_stored_info
0,5 Seconds of Summer,Easier,"[Outro: All, Luke]","I love you so much that I hate you (Hate you)\nRight now, it's so hard to blame you\n'Cause you're so damn beautiful",[Outro],"artist: 5 Seconds of Summer, title: Easier, tag: [Outro], content: I love you so much that I hate you (Hate you)\nRight now, it's so hard to blame you\n'Cause you're so damn beautiful"
1,5 Seconds of Summer,Easier,[Verse 1: Luke],"Why do we always gotta run away?\nAnd we wind up in the same place\nIt's like we're looking for the same thing\nSame thing, yeah\nYeah, do we really gotta do this now?\nRight here with all your friends around\nIn the morning, we can work it out\nWork it out",[Verse],"artist: 5 Seconds of Summer, title: Easier, tag: [Verse], content: Why do we always gotta run away?\nAnd we wind up in the same place\nIt's like we're looking for the same thing\nSame thing, yeah\nYeah, do we really gotta do this now?\nRight here with all your friends around\nIn the morning, we can work it out\nWork it out"
2,5 Seconds of Summer,Easier,[Bridge: Luke],The hardest part of all (Da-da-da-dum-da-da-da-da)\nIs that we're only built to fall (Is it easier?),[Bridge],"artist: 5 Seconds of Summer, title: Easier, tag: [Bridge], content: The hardest part of all (Da-da-da-dum-da-da-da-da)\nIs that we're only built to fall (Is it easier?)"
3,5 Seconds of Summer,Easier,"[Pre-Chorus: All, Luke]","I love you so much that I hate you\nRight now, it's so hard to blame you\n'Cause you're so damn beautiful\nSo damn beautiful",[Pre-Chorus],"artist: 5 Seconds of Summer, title: Easier, tag: [Pre-Chorus], content: I love you so much that I hate you\nRight now, it's so hard to blame you\n'Cause you're so damn beautiful\nSo damn beautiful"
4,5 Seconds of Summer,Easier,[Intro: Luke],"Is it easier to stay? Is it easier to go?\nI don't wanna know, oh\nBut I know that I'm never, ever gonna change\nAnd you know you don't want it any other way",[Intro],"artist: 5 Seconds of Summer, title: Easier, tag: [Intro], content: Is it easier to stay? Is it easier to go?\nI don't wanna know, oh\nBut I know that I'm never, ever gonna change\nAnd you know you don't want it any other way"


In [21]:
collection = train['db_stored_info'].to_list() + dev['db_stored_info'].to_list()

# ✏️ Database Generation

In [66]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

ImportError: cannot import name 'Indexer' from 'colbert' (/Users/mariaribalta/Documents/EHU/Applications_2/final_project/apps2_project_venv/lib/python3.9/site-packages/colbert/__init__.py)

In [ ]:
class Colbert_IR():
  def __init__(self, checkpoint: str, collection: list[str], index_name: str, doc_maxlen: int = 300, nbits: int = 2):
    self.nbits = nbits
    self.doc_maxlen = doc_maxlen
    self.index_name = index_name
    self.collection = collection
    self.checkpoint = checkpoint

    with Run().context(RunConfig(nranks=1, experiment='notebook')):
      config = ColBERTConfig(doc_maxlen=self.doc_maxlen, nbits=self.nbits, kmeans_niters=4)

      self.indexer = Indexer(checkpoint=self.checkpoint, config=config)
      self.indexer.index(name=self.index_name, collection=self.collection, overwrite=True)

    with Run().context(RunConfig(experiment='notebook')):
      self.searcher = Searcher(index=index_name, collection=collection)


  def retrieve_documents(self, query: str, topk:int=3):
    ids, ranks, scores = self.searcher.search(query, k=topk)
    passages = [self.searcher.collection[id] for id in ids]

    return passages, scores


In [ ]:
checkpoint = 'colbert-ir/colbertv2.0'
index_name = 'songs_index'

In [ ]:
colbert_ir = Colbert_IR(checkpoint, collection, index_name)

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]



[May 19, 13:43:29] #> Creating directory /content/experiments/notebook/indexes/songs_index 


#> Starting...
#> Joined...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[May 19, 13:48:19] #> Loading codec...
[May 19, 13:48:19] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[May 19, 13:48:19] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[May 19, 13:48:19] #> Loading IVF...
[May 19, 13:48:19] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1569.14it/s]

[May 19, 13:48:19] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 38.41it/s]


# 📄 Gemma Baseline Song Generation

In [75]:
test['prompt_baseline_gemma'] = test.apply(lambda row: f'Write the {row["tag_2"]} of a song titled "{row["song"]}" in the style of "{row["artist"]}". Generate only 4 sentences.', axis=1)

In [ ]:
client = Groq()

In [76]:
def ask_gemma(input_data: list[str], outfile: str, last_idx: int, max_rows: int):
  with open(outfile, 'w') as output_file:
      output_file.write('[')
      for i in range(last_idx, len(input_data[:max_rows])):
          try:
              sample = [{'role':'user', 'content': input_data[i]}]
              chat_completion = client.chat.completions.create(
                  messages=sample,
                  model="gemma-7b-it",
                  temperature=0.5,
                  max_tokens=1024,
                  top_p=1,
                  stop=None,
                  stream=False,
              )
              answer = chat_completion.choices[0].message.content

              output_sample = {'sentence': sample[-1]['content'],'generated_content': answer, 'id': i}

              output_file.write(json.dumps(output_sample))

              if i != (max_rows - 1):
                  output_file.write(',')
          except:
              print('next start by:')
              print(i + 1)
              break
      output_file.write(']')

In [87]:
last_idx = 195
max_rows = 196
outfile = f'output_gemma_baseline/{last_idx}_starting_row.json'
input_data = test['prompt_baseline_gemma'].to_list()

ask_gemma(input_data, outfile, last_idx, max_rows)

In [110]:
def recover_all_files(path: str):
    files_content = []

    for filename in os.listdir(path):
        if filename.endswith('.json'):
            with open(f"{path}/{filename}") as infile:
                file_content = json.load(infile)
                files_content += file_content
    return pd.DataFrame(files_content).sort_values(by="id").reset_index(drop=True)

In [111]:
gemma_baseline_files = recover_all_files('output_gemma_baseline')
gemma_baseline_df = pd.concat([test, gemma_baseline_files], axis=1)
assert all(gemma_baseline_df.index == gemma_baseline_df.id)

In [112]:
gemma_baseline_df.to_csv('output_gemma_baseline/gemma_baseline_generated_content.csv', index=False)

# 📝 Gemma RAG Song Generation

In [ ]:
test['prompt_baseline_gemma'] = test.apply(lambda row: f'Write the {row["tag_2"]} of a song titled "{row["song"]}" in the style of "{row["artist"]}". Generate only 4 sentences.', axis=1)

In [ ]:
client = Groq()

In [ ]:
class gemma_QA:
    def __init__(self, ir_class, groq_client, model):
        self.ir = ir_class
        self.client = groq_client
        self.model = model

    def template_builder(self, query, retrieved_info):
      return [{ "role": "user", "content": f"{query} You can get inspired by: {retrieved_info}. Make it 4 sentences only."}]

    def answer_question_batch(self, queries: list[str], outfile: str, last_idx: int, max_rows: int, topk:int=10,):
      with open(outfile, 'w') as output_file:
        output_file.write('[')
        for i in range(last_idx, len(input_data[:max_rows])):
            try:
                sample = queries[i]
                retrieved_info, scores = self.ir.retrieve_documents(sample, topk)
                retrieved_info = '\n'.join(retrieved_info)

                input = self.template_builder(sample, retrieved_info)

                chat_completion = client.chat.completions.create(
                    messages=input,
                    model=self.model,
                    temperature=0.5,
                    max_tokens=1024,
                    top_p=1,
                    stop=None,
                    stream=False,
                )

                answer = chat_completion.choices[0].message.content

                output_sample = {'sentence': sample, 'generated_content': answer, 'id': i}

                output_file.write(json.dumps(output_sample))

                if i != (max_rows - 1):
                    output_file.write(',')
            except:
                print('next start by:')
                print(i + 1)
                break
        output_file.write(']')

In [ ]:
def template_builder(query, retrieved_info):
  return [
    { "role": "user", "content": f" {query} Maybe this helps: {retrieved_info}. Make it short"},
  ]

In [ ]:
gemma_qa = gemma_QA(colbert_ir, client, "gemma-7b-it")

In [ ]:
last_idx = 0
max_rows = 962
outfile = f'{localpath}/output/{last_idx}_starting_row.json'
input_data = test['prompt_baseline_gemma'].to_list()

gemma_qa.answer_question_batch(input_data, outfile, last_idx, max_rows)

In [114]:
gemma_rag_files = recover_all_files('output_gemma_rag')

In [117]:
gemma_rag_files_df = pd.concat([test, gemma_rag_files], axis=1)
assert all(gemma_rag_files_df.index == gemma_rag_files_df.id)

In [118]:
gemma_rag_files_df

,artist,song,tag,content,tag_2,prompt_baseline_gemma,sentence,generated_content,id
0,5 Seconds of Summer,No Shame,[Verse 1: Luke],"Angel, with the gun in your hand\nPointin' my direction, givin' me affection\nLove is fatal, won't you give it a chance?\nCenter of attention, don't you ask me any questions",[Verse],"Write the [Verse] of a song titled ""No Shame"" in the style of ""5 Seconds of Summer"". Generate only 4 sentences.","Write the [Verse] of a song titled ""No Shame"" in the style of ""5 Seconds of Summer"". Generate only 4 sentences.","Moonlight paints the lonely night,\nMy reflection shows a fading light.\nHappiness, a distant memory's trace,\nIn this shadows, I find no solace in this space.",0
1,5 Seconds of Summer,No Shame,"[Pre-Chorus: Luke, Luke & Ashton]","Go on and light me like a cigarette\nEven if it might be somethin' you regret\nYou got me now, now, now\nSwallow me down, down, down, down",[Pre-Chorus],"Write the [Pre-Chorus] of a song titled ""No Shame"" in the style of ""5 Seconds of Summer"". Generate only 4 sentences.","Write the [Pre-Chorus] of a song titled ""No Shame"" in the style of ""5 Seconds of Summer"". Generate only 4 sentences.","Shadows linger, secrets hide,\nYour gaze like venom, a venomous tide.\nCan't deny the pull you hold,\nIn this twisted game, we lose control.",1
2,5 Seconds of Summer,No Shame,[Chorus: All],I only light up when cameras are flashin'\nNever enough and no satisfaction\nGot no shame\nI love the way you're screaming my name\nDiggin' my grave to get a reaction\nChangin' my face and callin' it fashion\nGot no shame\nI love the way you're screaming my name,[Chorus],"Write the [Chorus] of a song titled ""No Shame"" in the style of ""5 Seconds of Summer"". Generate only 4 sentences.","Write the [Chorus] of a song titled ""No Shame"" in the style of ""5 Seconds of Summer"". Generate only 4 sentences.","Now you're looking brand new, but I wish we'd never met\nCleanin' up your mess, while he's takin' off your dress, I forget\nIf I ask you if you love me, will you whisper in my ear?\nLie to me, lie, lie, lie, lie to me, my dear.",2
3,5 Seconds of Summer,No Shame,[Verse 2: Luke],"Lay me in the palm of your hand\nI'll give you my permission, you'll always be forgiven\nGo on, replace me\nWhen you're cravin' somethin' sweeter than the words I left in your mouth\nGo on and spit me out\nYou might also like",[Verse],"Write the [Verse] of a song titled ""No Shame"" in the style of ""5 Seconds of Summer"". Generate only 4 sentences.","Write the [Verse] of a song titled ""No Shame"" in the style of ""5 Seconds of Summer"". Generate only 4 sentences.","Moonlight paints the lonely night,\nShadows whisper secrets in the candlelight.\nI'm lost in this maze, my heart a hollow drum,\nBut I'll keep searching, 'cause I'm not done.",3
4,5 Seconds of Summer,No Shame,[Chorus: All],I only light up when cameras are flashin'\nNever enough and no satisfaction\nGot no shame\nI love the way you're screaming my name\nDiggin' my grave to get a reaction\nChangin' my face and callin' it fashion\nGot no shame\nI love the way you're screaming my name,[Chorus],"Write the [Chorus] of a song titled ""No Shame"" in the style of ""5 Seconds of Summer"". Generate only 4 sentences.","Write the [Chorus] of a song titled ""No Shame"" in the style of ""5 Seconds of Summer"". Generate only 4 sentences.","Now that you're out of sight, I'm feelin' alright\nBut I know you're takin' off in the moonlight tonight\nSo tell me one last time, before you fade from sight\nDo you love me, love me, love me in the night?",4
...,...,...,...,...,...,...,...,...,...
956,blackbear,​if i could i would feel nothing,[Chorus],"'Cause if I could I would feel nothing\nThat's the truth and I don't care, baby\nThat's the truth and I don't care\nMix the Henny with these pills\nThe world is cold and life's not fair, baby\nThe world is cold and life's not fair",[Chorus],"Write the [Chorus] of a song titled ""​if i could i would feel nothing"" in the styl

In [119]:
pd.concat([test, gemma_rag_files], axis=1).to_csv('output_gemma_rag/gemma_rag_generated_content.csv', index=False)

# 🤓 Results Analysis

In [ ]:
MISTRAL_API = os.getenv("MISTRAL_API")
client = MistralClient(api_key=MISTRAL_API)

In [ ]:
def generate_prompt(tag, lyrics):
    return f"""ONLY THE SCORE.
    Give me the score from 0/10 coherence, musicality and quality. 
    Example: 
    coherence X/10
    musicality y/10
    quality z/10
    of this {tag} of a pop song: 
    {lyrics}"""

In [ ]:
original_prompts = [
    generate_prompt(row["tag_2"], row["Original Lyrics"]) for _, row in data.iterrows()
]
generated_prompts = [
    generate_prompt(row["tag_2"], row["Generated Lyrics"]) for _, row in data.iterrows()
]
generated_prompts

In [ ]:
def send_message_mistral(
    client: MistralClient, message: str, mistral_model="open-mistral-7b"
) -> str:
    """
    Send a message to the Mistral API and return the response.

    Parameters:
        client: Mistral API client.
        message: The message to send to the API.

    Returns:
        str: Model response.
    """
    messages = [ChatMessage(role="user", content=message)]
    chat_response = client.chat(
        model=mistral_model,
        messages=messages,
    )
    return chat_response.choices[0].message.content

In [ ]:
original_eval = [send_message_mistral(client, prompt) for prompt in original_promts]
generated_eval = [send_message_mistral(client, prompt) for prompt in generated_promts]

In [ ]:
data["Generated_eval"] = generated_eval
data["Original_eval"] = original_eval
data

In [ ]:
import re

pattern = r"(\w+)\:\s+(\d+)\/(\d+)"
data["Generated_punct"] = data["Generated_eval"].map(
    lambda row: {x[0]: (float(x[1]), float(x[2])) for x in re.findall(pattern, row)}
)
data["Original_punct"] = data["Original_eval"].map(
    lambda row: {x[0]: (float(x[1]), float(x[2])) for x in re.findall(pattern, row)}
)

data.head()

In [ ]:
for data_set in ("Generated", "Original"):
    for att in ["coherence", "musicality", "quality"]:
        data[f"{data_set}_{att}"] = data[f"{data_set}_punct"].map(
            lambda x: 10 * x.get(att, (1, -1))[0] / x.get(att, (1, -1))[1]
        )

In [ ]:
for data_set in ("Generated", "Original"):
    data[f"{data_set}_mean"] = data[
        [f"{data_set}_{att}" for att in ["coherence", "musicality", "quality"]]
    ].mean(axis=1)
data.head()

In [ ]:
data.to_csv("../data/results_mistral_gemma.csv", index=False)

## Current Results with GEMMA+RAG